In [1]:
import pickle
from util.params import Params
from util.env import set_param
from main import Main
import torch
from models.GDN import GDN
from test_loop import test
from train_loop import train
import pandas as pd
from util.data import getAttacks
from evaluate import createMetrics, createStats
import pickle
import matplotlib.pyplot as plt
import os
import numpy
from ipywidgets import Checkbox, interact, FloatSlider, fixed


def plot_with_full_row(
    thr=False,
    pred=False,
    threshold=1.0,
    stat: dict = None,
    score: numpy.ndarray = None,
    labels: numpy.ndarray = None,
    run:int=0
):
    minimum = 0  # val_result[0].min()
    maximum = stat["threshold"].cpu().clone().numpy() * threshold  # *24.5
    pred_filtered = numpy.where((score > maximum), 1, 0)

    df = pd.DataFrame(
        {
            "thr": numpy.ones_like(pred_filtered) * maximum,
            "pred": score,
            "pred_filterd": pred_filtered,
            "truth": labels,
        }
    )
    attacks = getAttacks(df, "truth")
    num_plots = len(attacks)  # Number of data points
    num_rows = (num_plots + 1) // 2  # Two columns per row

    # Create a figure with a grid layout, including space for the full-row plot
    fig = plt.figure(figsize=(12, 6 + 5 * num_rows))  # Adjust overall size
    grid = fig.add_gridspec(num_rows + 1, 2)  # Add one row for the overarching plot

    # Full-row plot at the top
    ax_full_row = fig.add_subplot(grid[0, :])  # Span across all columns
    df.plot(ax=ax_full_row)
    ax_full_row.set_title(f"all attacks at  {run}")
    ax_full_row.legend()
    cols = df.columns
    if not thr:
        cols = [col for col in cols if col != "thr"]
    if not pred:
        cols = [col for col in cols if col != "pred"]
    max = df.shape[0]
    # Individual subplots below
    for i, attack in enumerate(attacks):
        leng = attack[1] - attack[0]
        left = attack[0] - leng
        right = attack[1] + leng
        right = right if right < max else max
        left = left if left > 0 else 0
        row, col = divmod(i, 2)  # Determine row and column for the subplot
        ax = fig.add_subplot(grid[row + 1, col])  # Offset by one for the full-row plot
        df.loc[left:right, cols].plot(ax=ax)  # Example plot
        ax.set_title(f"attack {i+1}")

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [9]:
main_path="./snapshot/gnn_tam_swat/25_04_11_17_15_19"
count=1

In [8]:
scores = []
labels = None
stats = []
labels=[]
for run in range(count):
    path = f"{main_path}/{run}/"
    if not os.path.exists(f"{path}param.pickle"):
        print(f"Skipping run {run}")
        continue
    param: Params = pickle.load(file=open(f"{path}param.pickle", "rb"))
    main.model.cpu()
    mode_dict = pickle.load(file=open(f"{path}model_parameters.pickle", "rb"))
    main.model.cuda()
    param.val_ratio = 0
    set_param(param)
    # print(param.summary())
    main = Main(param=param, modelParams=mode_dict)
    # main.model.load_state_dict(pickle.load(open(main.param.best_path().replace("best.pt","best_train.pt"),"rb")), weights_only=True)
    _, _, label = next(
            iter(
                torch.utils.data.DataLoader(
                    main.test_dataset,
                    batch_size=main.test_dataset.__len__(),
                    shuffle=False,
                    pin_memory=False,
                )
            )
        )
    labels.append(label.cpu().squeeze(-1).numpy())
    # print(len(main.train_dataloader),len(main.val_dataloader),len(main.test_dataloader))
    train_losses = test(main.model, main.train_dataloader)
    stat = createStats(train_losses)
    # print(stat)
    # print("train_avg_loss: ",train_avg_loss)
    # val_avg_loss, val_result = test(main.model, main.val_dataloader)
    # val_avg_loss
    test_losses = test(main.model, main.test_dataloader)

    # print("test_avg_loss: ",test_avg_loss)
    # print("max: ",test_result[0].max(),
    #     #   val_result[0].max(),
    #       train_result[0].max())
    # print("min: ",test_result[0].min(),
    #     #   val_result[0].min(),
    #       train_result[0].min())
    score = ((test_losses - stat["medians"]).abs() / stat["iqr"]).max(-1).values
    score=score.cpu().clone().numpy()
    print("score:\n", score.min(), score.max(), score.mean())
    scores.append(score)
    stats.append(stat)

print("FINISHED")

# DATASET 

*Datasets.swat*
Model is trained. Loading from file .....


score:
 1.1863654 220.41959 54.86868
FINISHED


In [4]:
for run in range(count):
    interact(
        plot_with_full_row,
        thr=Checkbox(value=True, description="thr"),
        pred=Checkbox(value=True, description="pred"),
        threshold=FloatSlider(value=1.0,min=0.5,max=2.0,step=.01,description="threshold"),
        score=fixed(scores[run]),
        stat=fixed(stats[run]),
        labels=fixed(labels[run]),
        run=fixed(run)
    )

interactive(children=(Checkbox(value=True, description='thr'), Checkbox(value=True, description='pred'), Float…

In [ ]:
from torchmetrics.classification import BinaryStatScores

selected_run=2
threshold=.91
for value in [1,threshold]:
    
    new_threshold=stats[selected_run]["threshold"].cpu().numpy()*value
    pred=numpy.where(scores[selected_run]>new_threshold,1,0)
    bc=BinaryStatScores()
    bc.update(torch.tensor(pred),torch.tensor(labels[selected_run]))
    cfm=bc.compute()
    print(createMetrics(cfm))